# Importing the metadata.csv file

In [1]:
import numpy as np
import pandas as pd 

metadata_df = pd.read_csv('metadata.csv')
metadata_df.head()

C:\Users\David\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,arxiv_id,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,unk,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125340?pdf=re...
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a ...",10.1093/emboj/21.9.2076,PMC125375,11980704.0,unk,CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc125375?pdf=re...
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,PMC,Synthesis of a novel hepatitis C virus protein...,10.1093/emboj/20.14.3840,PMC125543,11447125.0,no-cc,Hepatitis C virus (HCV) is an important human ...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; ...",EMBO J,NaN,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,PMC,Structure of coronavirus main proteinase revea...,10.1093/emboj/cdf327,PMC126080,12093723.0,unk,The key enzyme in coronavirus polyprotein proc...,2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, J...",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc126080?pdf=re...
4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,PMC,Discontinuous and non-discontinuous subgenomic...,10.1093/emboj/cdf635,PMC136939,12456663.0,unk,"Arteri-, corona-, toro- and roniviruses are ev...",2002-12-01,"van Vliet, A.L.W.; Smits, S.L.; Rottier, P.J.M...",The EMBO Journal,NaN,NaN,NaN,True,True,custom_license,http://europepmc.org/articles/pmc136939?pdf=re...


# Finding the number of records with missing abstract or doi

In [2]:
missing_values_count_per_col = metadata_df.isnull().sum()
print('Entries without abstract BEFORE: '+ str(missing_values_count_per_col['abstract']))
print('Entries without doi BEFORE: '+ str(missing_values_count_per_col['doi']))

Entries without abstract BEFORE: 11130
Entries without doi BEFORE: 4086


# Visualizing the percentage of missing values in each column in the metadata.csv file

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

missing_values_count_per_col = metadata_df.isnull().sum()/len(metadata_df) * 100
missing_values_count_per_col = missing_values_count_per_col.to_frame()
missing_values_count_per_col.columns = ['Percentage of Missing Values']
missing_values_count_per_col.index.names = ['Column Name']
missing_values_count_per_col['Column Name'] = missing_values_count_per_col.index

sns.set(style="whitegrid", color_codes=True)
sns.barplot(x = 'Column Name', y = 'Percentage of Missing Values', data=missing_values_count_per_col)
plt.xticks(rotation = 90)
plt.show()

<Figure size 640x480 with 1 Axes>

# Dropping records with missing abstract, doi or title

In [4]:
#Drop rows with missing abstract or doi
metadata_df = metadata_df.dropna(how='any', subset=['abstract','doi', 'title'])
missing_values_count_per_col = metadata_df.isnull().sum()
print('Entries without abstract AFTER: '+ str(missing_values_count_per_col['abstract']))
print('Entries without doi AFTER: '+ str(missing_values_count_per_col['doi']))

Entries without abstract AFTER: 0
Entries without doi AFTER: 0


In [5]:
# Select all duplicate rows based on DOI
duplicateRowsDF = metadata_df[metadata_df.duplicated(['doi'])]
print("Duplicate rows based on DOI: " + str(duplicateRowsDF['doi'].count()))
metadata_df = metadata_df.drop_duplicates(subset='doi', keep="first")
metadata_df = metadata_df.reset_index(drop=True)

Duplicate rows based on DOI: 12


In [8]:
from nltk import word_tokenize,sent_tokenize
import re
#from nltk.stem import PorterStemmer
#stemmer = PorterStemmer()
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
nlp = English()

def tokenize_and_remove_stopwords(text):
    tokens = [word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    tokens_after_stopwords_removal = []
    for token in filtered_tokens:
        lexeme = nlp.vocab[token]
        if not lexeme.is_stop:
            tokens_after_stopwords_removal.append(token)
    return tokens_after_stopwords_removal

## Removing the stopwords from the abstracts ##

In [11]:
clean_abstracts = []
for i in range(0, metadata_df.shape[0]):
    abstract = metadata_df.iloc[i]['abstract']
    clean_abstracts.append(tokenize_and_remove_stopwords(abstract))

In [13]:
import gensim
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

labelled_abstracts=[]
for i in range(0, len(clean_abstracts)):
    labelled_abstracts.append(TaggedDocument(clean_abstracts[i], "doc: " + str(i)))
print(labelled_abstracts[1])    

TaggedDocument(['CEACAM1', 'member', 'carcinoembryonic', 'antigen', 'CEA', 'family', 'Isoforms', 'murine', 'CEACAM1', 'serve', 'receptors', 'mouse', 'hepatitis', 'virus', 'MHV', 'murine', 'coronavirus', 'report', 'crystal', 'structure', 'soluble', 'murine', 'sCEACAM1a', 'composed', 'Ig-like', 'domains', 'MHV', 'neutralizing', 'activity', 'N-terminal', 'domain', 'uniquely', 'folded', 'CC′', 'loop', 'encompasses', 'key', 'virus-binding', 'residues', 'atomic', 'structure', 'member', 'CEA', 'family', 'provides', 'prototypic', 'architecture', 'functional', 'exploration', 'CEA', 'family', 'members', 'discuss', 'structural', 'basis', 'virus', 'receptor', 'activities', 'murine', 'CEACAM1', 'proteins', 'binding', 'Neisseria', 'human', 'CEACAM1', 'homophilic', 'heterophilic', 'interactions', 'CEA', 'family', 'members'], doc: 1)


In [14]:
model = Doc2Vec(labelled_abstracts)

In [15]:
for epoch in range(10):
    model.train(labelled_abstracts,epochs=model.iter,total_examples=model.corpus_count)
    print("Epoch #{} is complete.".format(epoch+1))

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


Epoch #1 is complete.
Epoch #2 is complete.
Epoch #3 is complete.
Epoch #4 is complete.
Epoch #5 is complete.
Epoch #6 is complete.
Epoch #7 is complete.
Epoch #8 is complete.
Epoch #9 is complete.
Epoch #10 is complete.


In [23]:
model.most_similar('incubation')
#model.docvecs.most_similar("2")
print(model)

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


Doc2Vec(dm/m,d100,n5,w5,mc5,s0.001,t3)


In [ ]:
covid19_df = metadata_df[['title', 'doi', 'abstract', 'publish_time', 'authors', 'journal', 'url']].copy()
questions_strings = []
q1 = "range of incubation periods for the disease in humans and how this varies across age and health status and how long individuals are contagious even after recovery"
q2 = "physical science of the coronavirus charge distribution adhesion to hydrophilic phobic surfaces environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding"
questions_strings.append(q1)
questions_strings.append(q2)
questions_lists = []

#Removing the stopwords and tokenizing the queries.
for i in range(0, len(questions_strings)):
    tokens_after_stopwords_removal = []
    question = questions_strings[i].split()
    for token in question:
        lexeme = nlp.vocab[token]
        if not lexeme.is_stop:
            tokens_after_stopwords_removal.append(token)
    questions_lists.append(tokens_after_stopwords_removal)

wordfreq_total = [] #Contains TF counts of the queries in questions_lists in order.
for i in range(0, len(questions_lists)):
    wordfreq = []
    for w in questions_lists[i]:
        wordfreq.append(questions_lists[i].count(w))
    wordfreq_total.append(wordfreq)

#Normalizing the TF counts of the queries using the L2 norms method
for i in range(0, len(wordfreq_total)):
    norm_value = 0
    for w in wordfreq_total[i]:
        norm_value += w**2
    print(wordfreq_total[i])
    print(norm_value)

In [ ]:
import gensim 
from gensim.models import Word2Vec

covid19_df = metadata_df[['title','abstract']]
covid19_df2 = covid19_df.apply(lambda x: '#'.join(x.astype(str)), axis=1)
df_info = pd.DataFrame({'info': covid19_df2})
sent = [row.split('#') for row in df_info['info']]
model = Word2Vec(sent, min_count = 1, size = 100, window = 5)
sent[0:3]

In [ ]:
model.similarity('Detection of hepatitis C virus in the nasal secretions of an intranasal drug-user'
                 , 'Synthesis of a novel hepatitis C virus protein by ribosomal frameshift')

In [ ]:
from wordcloud import WordCloud
from nltk.corpus import stopwords
import re 

stop_words = set(stopwords.words('english'))
print(stop_words)
new_stopwords = ['using, use, chapter']
new_stopwords_list = stop_words.union(new_stopwords)
text = " ".join(str(title) for title in covid19_df.title)
text = text.lower()

wordcloud = WordCloud(regexp= r'\w[\w-]+', stopwords=new_stopwords_list, max_font_size=50, max_words=100, background_color="white").generate(text)
plt.figure(figsize=[12,6])
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show() 
